In [2]:
import numpy as np

# fast PCP 
# inpout is D -> m*n, lambda, tau
# rank is defined for parameter of decomposition
# we derived the thin SVD from original SVD 
import numpy as np 
import math

# the size of vector could be 1 x n or n x 1

def shrinkage(x, para):
    return np.sign(x) * max(0, abs(x)-para)

def fastPCP(r, tau, lam_bda):
    S = np.zeros((1080, 1920))
    # rank of SVD
    r = 10
    for i in range(0, m):
        left, singular, right = svd_res(imgGray - S, r)
        L = np.matmul(np.matmul(left, singular),right)
        if singular[r]/sum(singular) > tau:
            r += 1
        S = shrinkage(imgGray - L, lam_bda)

def incSVD(D):
    # first time SVD
    l0,dt0,r0 = svd_res(D, rank) #dt0 is r * r 
    m = D.shape[0]
    l = D.shape[1]
    rank = r
    d_vector = np.zeros((m, 1)) # d is a vector with m rows and 1 column
    e_vector = np.zeros((1, l+1))
    # get the result of D_canc
    D_canc = numpy.concatenate(D, d_vector)
    # second time thin SVD
    l1,dt1,r1 = svd_res(D_canc, rank)
    # gran-schmidt orthonormalization
    z = d_vector - np.matmul(np.matmul(l, np.transpose(l)),d_vector)
    x = np.matmul(np.transpose(l0), d_vector) # x is r * 1
    rho_x = np.linalg.norm(x, ord=2) # single value 
    p = 1/rho_x * z 
    zero_T = np.zeros((1, r)) # zeroT is 1 * r
    # input with original paper (20)
    dim1 = np.concatenate((dt0, x), axis=1)
    dim2 = np.concatenate((zeroT, rho_x), axis=1)
    dim = np.concatenate((dim1, dim2), axis=0)
    # third time SVD with original paper(20)
    G, res_dt, HofT = svd_res(dim, new_rank)
    U1 = np.matmul(l0,G[1:r, 1:r]) + np.matmul(p,G[r+1,1:r]) # result should be m * r
    res_dt1 = res_dt[1:r, 1:r]
    V1_1 = np.matmul(np.transpose(r0), np.transpose(HofT)) # l * r
    V1_2 = np.matmul(r0, np.transpose(HofT)) # (r+1) * r
    V1 = np.concatenate((V1_1, V2_1), axis=0)
    return U1, dt1, V1
    

def dwnSVD(D):
    # the size of D is m*n
    # the size of d_vector is m*1
    d_vector = np.zeros((m, 1))
    D_canc = numpy.concatenate((D, d_vector), axis = 1)
    # the value of e is 1/(l+1)
    e_vector = np.zeros((1, l+1))
    # size of v0 is (l+1, r)
    u0, sing0, v0 = svd_res(D_canc, rank)
    y = np.transpose(v0[l+1,:]) # size of y is r * 1
    # the shape of z is 1 * l or 1 * (l+1) which is a vector 
    z = e_vector - np.matmul(v0, y)
    rho_y = math.sqrt(1- np.matmul(np.transpose(y), y))
    # size is (r+1) * (r+1)
    left_corner = np.matmul(sing0-np.matmul(np.matmul(sing0, y), np.transpose(y)))
    # size of q is n*1 
    q = 1/rho_y * z
    right_corner = -rho_y * np.matmul(sing0, y)
    left_footer = np.zeros((1, rank))
    right_footer = 0
    dim1 = np.concatenate((left_corner, right_corner), axis = 1)
    dim2 = np.concatenate((left_footer, right_footer), axis = 1)
    dim = np.concatenate((dim1, dim2), axis = 0)
    G, res_dt, HofT = svd_res(dim, new_rank)
    # the result of U1 is m*r
    U1 = np.matmul(u0, G[1:r, 1:r])
    # the result of dt1 is r * r
    dt1 = res_dt[1:r, 1:r]
    # the result of V1 is n*r
    V1 = np.matmul(v0, np.transpose(HofT)[1:r, 1:r]) + np.matmul(q, np.transpose(HofT)[r+1, 1:r])
    return U1, dt1, V1
    

def repSVD(D):
    # the size of D is m*l 
    # the size of d is m*1 
    d_vector = np.zeros((m, 1))
    e_vector = np.zeros((1, l+1))
    D_canc = numpy.concatenate((D, d_vector), axis=1)
    u0, sing0, v0 = svd_res(D_canc, rank)
    # size of m is r*m x m*1 = r x 1 
    x = np.matmul(np.transpose(u0), d_vector)
    # size of z_x is r x 1
    z_x = d_vector - np.matmul(u0, x)
    rho_x = np.linalg.norm(z_x, ord=2)
    
    y = np.matmul(v0, e_vector)
    z_y = e_vector - np.matmul(np.transpose(v0),y)
    rho_y = np.linalg.norm(z_y, ord=2) 
    
    p = z_x/rho_x
    q = z_x/rho_y
    

# implementation of inc PCP 
def incPCP(D, regul_para, num_inner, back_frame, m):
    print(D[:, 1:k0].shape)
    # L + S = D[:, 1:k0]
    k = rank = 1
    left, sing, right = svd_res(L, r) # m*r, r*r, n*r
    for k in range(k0 + 1, n):
        m += 1
        left, sing, right = incSVD(D[:, k], left, sing, right)
        for j in range(1, num_inner):
            L[:k] = np.matmul(np.matmul(left[:, 1:r], sing), right[end,:])
            S[:k] = shrinkage(D[:,k] - L[:,k], regul_para)
            if j == num_inner:
                break
            left, sing, right = repSVD(D[:,k], S[:,k], left, sign, right)
    if m >= back_frame:
        dwnSVD("1st", left, sing, right)
    if np.linalg.norm(L[:, k]-L[:,k-1], ord = 2)**2/np.linalg.norm(L[:,k-1], ord=2)**2 >= tau:
        m = k0

In [3]:
# fast PCP 
# inpout is D -> m*n, lambda, tau
# rank is defined for parameter of decomposition
# we derived the thin SVD from original SVD 
import numpy as np 
import math
import random

# the size of vector could be 1 x n or n x 1

def shrinkage(x, para):
    for i in range(0, x.shape[0]):
        for j in range(0, x.shape[1]):
            x[i][j] = np.sign(x[i][j]) * max(0, abs(x[i][j])-para)
    return x
    

def fastPCP(r, tau, lam_bda):
    S = np.zeros((1080, 1920))
    # rank of SVD
    r = 10
    for i in range(0, m):
        left, singular, right = svd_res(imgGray- S, r)
        L = np.matmul(np.matmul(left, singular),right)
        if singular[r]/sum(singular) > tau:
            r += 1
        S = shrinkage(imgGray - L, lam_bda)

'''
# incremental SVD implementation 
# author: sarah wang
'''   
def incrementalSVD(D, rank): # updateSVD
    # first time SVD
    #r = 200
    r = rank
    l0,dt0,r0 = svd_res(D, rank) #dt0 is r * r 
    m = D.shape[0]
    l = D.shape[1]
    # !!!!!the value of d_vector is not zeros
    random_m = random.randint(0, m)
    random_m = 0
    random_l = random.randint(0, l+1) 
    random_l = 0
    # size of d is m * 1
    # d is a vector with m rows and 1 column   
    d_vector = D[:, random_l]
    # second time thin SVD
    # get the result of D_canc
    D_canc = np.concatenate((D, d_vector.reshape(m, 1)), axis=1)
    # size of e is 1 * (l+1) or (l+1) * 1
    e_vector = D_canc[random_m,:]
    l1,dt1,r1 = svd_res(D_canc, rank)
    # gran-schmidt orthonormalization
    z = d_vector - np.matmul(np.matmul(l0, np.transpose(l0)),d_vector)
    x = np.matmul(np.transpose(l0), d_vector) # x is r * 1
    rho_x = np.linalg.norm(x, ord=2) # single value 
    p = 1/rho_x * z 
    zeroT = np.zeros((1, r)) # zeroT is 1 * r
    # input with original paper (20)
    dt0 = conv_diagnal(dt0,rank)
    x = x.reshape(rank, 1)
    dim1 = np.concatenate((dt0, x), axis=1)
    dim2 = np.append(zeroT, rho_x)
    dim = np.row_stack((dim1, dim2))
    # third time SVD with original paper(20)
    # new_rank here should be less than rank
    new_rank = rank
    G, res_dt, HofT = svd_res(dim, rank)
    #print(p.shape)
    U1 = np.matmul(l0,G[0:r, 0:r]) + np.matmul(p.reshape(m, 1), G[r,0:r].reshape(1,r)) # result should be m * r
    res_dt1 = conv_diagnal(res_dt, rank)[0:r, 0:r]
    V1_1 = np.matmul(np.transpose(r0), np.transpose(HofT)[0:r, 0:r]) # l * r
    V1_2 = np.transpose(HofT)[r,0:r] # (r+1) * r
    #print(V1_1.shape)
    #print(V1_2.shape)
    V1 = np.row_stack((V1_1, V1_2))
    return U1, dt1, V1

'''
# Downdating SVD implementation 
# author: sarah wang
'''       
def downdateSVD(D, rank): #downdateSVD
    # the size of D is m*n
    # the size of d_vector is m*1
    m = D.shape[0]
    l = D.shape[1]
    random_m = random.randint(0, m)
    random_m = 0
    random_l = random.randint(0, l+1) 
    random_l = 0
    d_vector = D[:, random_l].reshape(m, 1)
    D_canc = np.concatenate((D, d_vector), axis = 1)
    # the size of e_vector is l+1 * 1 or 1 * l+1
    e_vector = D_canc[random_m,:]
    # size of v0 is (l+1, r)
    #rank = 300
    u0, sing0, v0ofT = svd_res(D_canc, rank)
    y = np.transpose(v0ofT)[l,:] # size of y is r * 1
    # the shape of z is 1 * l or 1 * (l+1) which is a vector  
    z = e_vector.reshape(l+1,1) - np.matmul(np.transpose(v0ofT), y).reshape(l+1, 1)
    print(e_vector.shape)
    rho_y = np.linalg.norm(z, ord=2)
    # size is (r+1) * (r+1)
    y = y.reshape(rank, 1)
    left_corner = conv_diagnal(sing0,rank) + np.matmul(conv_diagnal(sing0, rank), np.matmul(y, np.transpose(y)))
    # size of q is l*1 
    q = (1/rho_y) * z
    right_corner = -rho_y * np.matmul(conv_diagnal(sing0, rank), y)
    left_footer = list(np.zeros((1, rank)))
    right_footer = 0
    dim1 = np.concatenate((left_corner, right_corner), axis = 1)
    dim2 = np.append(left_footer, right_footer)
    dim = np.row_stack((dim1, dim2))
    #print(q.shape)
    G, res_dt, HofT = svd_res(dim, rank)
    # the result of U1 is m*r
    U1 = np.matmul(u0, G[0:rank, 0:rank])
    # the result of dt1 is r * r
    dt1 = conv_diagnal(res_dt, rank)[0:rank, 0:rank]
    # the result of V1 is l*r
    #print(HofT.shape)
    V1 = np.matmul(np.transpose(v0ofT), np.transpose(HofT)[0:rank, 0:rank]) + np.matmul(q, np.transpose(HofT)[rank, 0:rank].reshape(1, rank))
    return U1, dt1, V1

In [4]:
'''
# Replacing SVD implementation 
# author: sarah wang
'''        

def replaceSVD(D, rank): #replaceSVD
    # the size of D is m*l 
    # the size of d is m*1 or 1*m 
    r = rank
    m = D.shape[0]
    l = D.shape[1]
    #  index 4 is out of bounds for axis 1 with size 4
    random_m = random.randint(0, m)
    random_m = 0
    random_l = random.randint(0, l+1)
    random_l = 0
    d_vector = D[:, random_l].reshape(m, 1)
    # the size of e is r * 1
    # r = rank = 600
    D_canc = np.concatenate((D, d_vector), axis=1)
    e_vector = D_canc[random_m,:]
    u0, sing0, v0ofT = svd_res(D_canc, rank)
    # size of x is r*m x m*1 = r x 1 
    x = np.matmul(np.transpose(u0), d_vector)
    # size of z_x is m x 1
    z_x = d_vector - np.matmul(u0, x)
    rho_x = np.linalg.norm(z_x, ord=2)
    # size of y is r x 1
    y = np.transpose(v0ofT)[l,:].reshape(rank, 1)
    # size of z_y is 1921 *  1
    z_y = e_vector.reshape(l+1, 1) - np.matmul(np.transpose(v0ofT),y)
    rho_y = np.linalg.norm(z_y, ord=2) 
    # size of p should be m x 1
    p = z_x/rho_x
    # size of q should be l x 1
    #print(z_y.shape)
    q = z_y/rho_y
    left_corner = conv_diagnal(sing0, rank) + np.matmul(x.reshape(rank, 1), np.transpose(y))
    right_corner = - rho_y * x # the size should be r * 1
    left_footer =  rho_y * np.transpose(y)
    right_footer = rho_x * rho_y
    dim1 = np.concatenate((left_corner, right_corner), axis = 1)
    dim2 = np.append(left_footer, right_footer)
    dim = np.row_stack((dim1, dim2))
    G, res_dt, HofT = svd_res(dim, rank)
    U1 = np.matmul(u0, G[0:r, 0:r]) + np.matmul(p.reshape(m,1), G[r, 0:r].reshape(1, r))
    dt1 = conv_diagnal(res_dt, rank)[0:r, 0:r]
    V1 = np.matmul(np.transpose(v0ofT), np.transpose(HofT[0:r, 0:r])) + np.matmul(q.reshape(l+1, 1), np.transpose(HofT[r-1, 0:r]).reshape(1, r)) # the size should be l*r
    return U1, dt1, V1            

In [5]:
left, singular, right = downdateSVD(imgGray, 3)

im_output_low_rank = Image.fromarray(singular+255)
if im_output_low_rank.mode != 'RGB':
    im_output_low = im_output_low_rank.convert('RGB')
im_output_low.save("./res/singular_incremental.png")

left, singular, right = replaceSVD(imgGray, 30)
im_output_low_rank = Image.fromarray(left * 255)
if im_output_low_rank.mode != 'RGB':
    im_output_low = im_output_low_rank.convert('RGB')
im_output_low.save("./left_incremental.png")

from skimage import io
'''
img = io.imread('./survelliance/0325.jpg')
img_res = convert_RGB(img)
imgGray = color.rgb2gray(img_res)
'''

left, singular, right = incrementalSVD(imgGray, 4)

import sklearn.decomposition
import io
import time

# implementation of inc PCP 
# lambda is regularization parameter 
def incPCP(D, lamb, num_inner, back_frame, m):
    # k is in the range of (2, r)
    m = k0 = D.shape[0] # 1080
    n = D.shape[1] # 1920
    L = np.zeros((m,n))
    S_input = np.zeros((m,n))
    r = rank = 1
    U, S, V = svd_res(L, rank)
    k0 = 1
    # D[:, 1:k0] size is m * k0
    input_matrix = D[:, 1: k0]
    left, sing, right = svd_res(L, r) # m*r, r*r, n*r
    sing = conv_diagnal(sing, r)
    time_start = time.time()
    count = 0
    for k in range(k0 + 1, n):
        count += 1
        print("current is the " + str(count) + " iteration")
        ++m
        #left, sing, right = incSVD(D[:,k], left_last, sing_last, right_last)
        ui, si, vi = incrementalSVD(D[:,0:k].reshape(m, k), rank)
        for j in range(1, num_inner+1): 
            L[:,0:k] = np.matmul(np.matmul(left[:, 0:r], conv_diagnal(sing, r)), right[-1,:].reshape(1,n))[:, 0:k] # the size of right side is 1080 * 1920
            # size of S is m * k
            S_input[:, 0:k] = shrinkage((D[:,0:k] - L[:,0:k]), lamb)
            if j == num_inner:
                break
            #left, sing, right = repSVD(D[:,k], S[:,k],E[:,k],left, sign, right)
            ur, sr, vr = replaceSVD(D[:,k].reshape(m, 1), rank)
    if m >= back_frame:
        #dwnSVD("1st", left, sing, right)
        ud, sd, vd = downdateSVD(D[:k], rank)
    # eigenvalue tolerance
    tau = 1e-4
    if np.linalg.norm(L[:, k]-L[:,k-1], ord = 2)**2/np.linalg.norm(L[:,k-1], ord=2)**2 >= tau:
        m = k0
    time_end = time.time()
    print("running time: " + str(time_end - time_start))
    return ud, sd, vd

left_res, diag_res, right_res = incPCP(imgGray, 10, 2, 30,imgGray.shape[0])

NameError: name 'imgGray' is not defined